<a href="https://colab.research.google.com/github/Drglazizzo/FreeCodeCamp-Machine-Learning-Project/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling



# Set seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Data Preprocessing
print("\nPreprocessing data...")

# Convert categorical data to numeric (one-hot encoding)
categorical_cols = ['sex', 'smoker', 'region']
numeric_cols = ['age', 'bmi', 'children']

dataset = pd.get_dummies(dataset, prefix='', prefix_sep='', columns=categorical_cols)
print("\nDataset after one-hot encoding:")
dataset.tail()

In [ ]:
# Split the data into train and test sets (80/20 split)
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print(f"\nTraining samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Separate numeric features for normalization
train_numeric = train_dataset[numeric_cols]
test_numeric = test_dataset[numeric_cols]

# Separate features and labels
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
# Normalize the numeric features
print("\nNormalizing numeric features...")
normalizer = layers.Normalization(input_shape=[3,])
normalizer.adapt(np.array(train_numeric))

In [ ]:
# Model Definition
print("\nBuilding model...")

def build_and_compile_model(norm):
    model = keras.Sequential([
        # Input layer for all features
        layers.Input(shape=(11,)),  # 11 features after one-hot encoding

        # Custom layer to apply normalization only to numeric features
        layers.Lambda(lambda x: tf.concat([
            norm(x[:, :3]),  # Normalize first 3 numeric features
            x[:, 3:]         # Pass through categorical features unchanged
        ], axis=1)),

        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(loss='mean_absolute_error',
                optimizer=keras.optimizers.Adam(0.001),
                metrics=['mae', 'mse'])
    return model

model = build_and_compile_model(normalizer)
print("\nModel summary:")
model.summary()

In [ ]:
# Model Training
print("\nTraining model...")
history = model.fit(
    train_dataset,
    train_labels,
    validation_split=0.2,
    verbose=1,
    epochs=100)

In [ ]:
# Plot training history
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, 10000])
plt.xlabel('Epoch')
plt.ylabel('Error [expenses]')
plt.title('Training History')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
